In [4]:
from qtpg.team import Team
from qtpg.agent import Agent
from qtpg.trainer import Trainer
from qtpg.figure13 import Figure13
import numpy as np

In [5]:
# gp params
numLearners = 4
numAgents = 20
numRuns = 1
numGens = 100
numEpisodes = 50
gap = 0.3
# rl params
alpha, discount, epsilon = 0.9, 0.9, 0.1
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0#-0.01
goalReached = 100

In [6]:
for run in range(numRuns):
    # init agent population
    trainer = Trainer(gap, numLearners, numAgents, alpha, discount, epsilon)
    
    # init env
    env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
    env.memoryRepeat = 0.0
    # search space to make rules
    trainer.generateRules(env)
    env.memoryRepeat = -0.01    
    trainer.createInitAgents()

    champs = []
    for gen in range(numGens):
        print(gen)
        env.reset()
        for agent in trainer.agents:
            print(agent.win)
            env.reset()
            episode_fitness = [] # new
            for ep in range(numEpisodes):
                if agent.win is not True:
                    env.reset()
                    fitness, win, states, seq = agent.evaluate_fitness(env)
                    episode_fitness.append(fitness)
#                     print(agent.team.q_table)
                    if win == True:
                        print('win stuff start')
                        print(states)
                        agent.win = True
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        for i in range(5):
                            env.reset()
                            agent.team.victory_lap(env, seq)
                        print('\n\n\n')
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        env.reset()
#                         agent.team.epsilon = 0.0
                        score, win, states = agent.replay(env)
                        print(agent.team.q_table)
                        print('win stuff end!')
                        print('REPLAY')
                        print(score)
                        print(win)
                        if not win:
                            agent.win = False
                        else:
                            champs.append(agent)
                            print('win replay!')
                        
                        print(states)
                        print('!!!!')
            # average fitness over episodes...
            average_fitness = 0
            average_fitness = np.mean(episode_fitness)
            print(average_fitness)
            agent.fitness = average_fitness
        trainer.evolve()

(0, 1)
2
0.1
[0, 0, 0, 0]
-1
[1, -1, -1, -1]
reward was positive
(0, 1)



(0, 1)
2
0.1
[0, 0, 0, 1]
-1
[1, -1, -1, -1]
reward was positive
(0, 2)



(0, 2)
2
0.1
[0, 0, 0, 2]
-1
[1, -1, -1, -1]
reward was positive
(0, 3)



(0, 3)
2
0.1
[0, 0, 0, 3]
-1
[1, -1, -1, -1]
reward was positive
(0, 4)



(0, 4)
2
0.1
[0, 0, 0, 4]
-1
[1, -1, -1, -1]
reward was negative, out of bounds of previous learner



(0, 4)
0
-0.01
[1, 4, 0, 0]
e8d938d0-0640-4bbd-8033-3eb378524b41
[0, 0, 0, 4]
reward was negative, in bounds of previous learner
curr: (0, 3)



(0, 3)
0
-0.01
[1, 4, 0, 0]
e8d938d0-0640-4bbd-8033-3eb378524b41
[0, 0, 0, 3]
reward was negative, in bounds of previous learner
curr: (0, 2)



(1, 2)
0
0.1
[1, 4, 0, 0]
e8d938d0-0640-4bbd-8033-3eb378524b41
[0, 0, 0, 2]
reward was positive
(2, 2)



(2, 2)
0
0.1
[1, 2, 0, 2]
e8d938d0-0640-4bbd-8033-3eb378524b41
[0, 0, 0, 2]
reward was positive
(3, 2)



(3, 2)
0
0.1
[1, 2, 0, 3]
e8d938d0-0640-4bbd-8033-3eb378524b41
[0, 0, 0, 2]
reward was positive





c3aa5b33-608a-426c-92bf-799768c6f12f 0 71.76619300185439
c3aa5b33-608a-426c-92bf-799768c6f12f 1 59.9906658710497
9d572c38-315e-4af8-a8d0-b967a15de078 1 88.49778789034127
9d572c38-315e-4af8-a8d0-b967a15de078 0 0
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 1 100.1
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 0 0
9d572c38-315e-4af8-a8d0-b967a15de078 3 70.07966866861206
9d572c38-315e-4af8-a8d0-b967a15de078 2 66.04135106118197
134ec05b-2aa6-4d54-a33c-72fefa328a17 1 0
134ec05b-2aa6-4d54-a33c-72fefa328a17 0 0
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 1 100.1
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 0 0
095079b1-d416-4031-960b-1ab7d859233b 2 0
095079b1-d416-4031-960b-1ab7d859233b 3 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 2 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 3 0
9fdef5a7-3059-4d2c-8e1b-c9e93fc3e864 1 0
9fdef5a7-3059-4d2c-8e1b-c9e93fc3e864 0 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 3 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 2 0
aa55f848-6a21-45c0-b473-414dd31579f2 3 0
aa55f848-6a21-45c0-b473-414dd31579f2 2 71.177815

win stuff start
[(0, 0), (0, 1), (0, 2), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 2), (3, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (3, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (3, 2), (4, 2), (4, 2), (4, 2), (3, 2), (4, 2), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 3), (4, 4), (4, 4), (4, 4), (4, 4), (4, 4), (4, 4), (4, 4), (4, 4), (3, 4), (2, 4)]
c3aa5b33-608a-426c-92bf-799768c6f12f 0 73.13349215918892
c3aa5b33-608a-426c-92bf-799768c6f12f 1 59.9906658710497
9d572c38-315e-4af8-a8d0-b967a15de078 1 88.6631203331854
9d572c38-315e-4af8-a8d0-b967a15de078 0 81.15916878014959
87a6bc18-8b7a

10.057599999999999
False
9.240999999999984
False
2.332999999999997
False
9.676599999999983
False
9.502799999999985
False
-0.08700000000000045
False
8.836199999999986
1
False
win stuff start
[(0, 0), (0, 1), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (1, 2), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 2), (4, 3), (4, 3), (4, 4), (3, 4), (2, 4)]
c3aa5b33-608a-426c-92bf-799768c6f12f 0 73.22776420521359
c3aa5b33-608a-426c-92bf-799768c6f12f 1 62.66584678430223
9d572c38-315e-4af8-a8d0-b967a15de078 1 90.18999847312033
9d572c38-315e-4af8-a8d0-b967a15de078 0 81.16086258082998
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 1 100.1
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 0 0
9d572c38-315e-4af8-a8d0-b967a15de078 3 73.24195318580763
9d572c38-315e-4af8-a8d0-b967a15de078 2 81.27029867706088
134ec05b-2aa6-4d54-a33c-72fefa328a17 





c3aa5b33-608a-426c-92bf-799768c6f12f 0 73.1280163982158
c3aa5b33-608a-426c-92bf-799768c6f12f 1 65.54894328319232
9d572c38-315e-4af8-a8d0-b967a15de078 1 90.18998252467405
9d572c38-315e-4af8-a8d0-b967a15de078 0 81.16084272206649
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 1 100.1
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 0 0
9d572c38-315e-4af8-a8d0-b967a15de078 3 73.24195318580763
9d572c38-315e-4af8-a8d0-b967a15de078 2 81.27021179642077
134ec05b-2aa6-4d54-a33c-72fefa328a17 1 0
134ec05b-2aa6-4d54-a33c-72fefa328a17 0 0
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 1 100.1
87a6bc18-8b7a-404c-a33d-c5ae5cf0f74e 0 0
095079b1-d416-4031-960b-1ab7d859233b 2 0
095079b1-d416-4031-960b-1ab7d859233b 3 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 2 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 3 0
9fdef5a7-3059-4d2c-8e1b-c9e93fc3e864 1 0
9fdef5a7-3059-4d2c-8e1b-c9e93fc3e864 0 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 3 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 2 0
aa55f848-6a21-45c0-b473-414dd31579f2 3 64.29143811327353
aa55f848-6a21-45

aa5b39fc-0108-473e-afe8-63076fd21bb9 3 0
aa5b39fc-0108-473e-afe8-63076fd21bb9 2 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 2 0
c7a4b146-1531-4b00-af58-5aef32ca99b2 3 0
0e1521a2-0b2e-49b8-bf4c-bf650a2cace4 0 0
0e1521a2-0b2e-49b8-bf4c-bf650a2cace4 1 0
f3221353-d052-4f2c-9738-565b40ae411d 0 0
f3221353-d052-4f2c-9738-565b40ae411d 1 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 0 0
400fe787-a42f-4ae4-98f1-f5159a4500c9 1 0
82ef65df-65f1-48e2-bee8-08e3e643186f 3 0
82ef65df-65f1-48e2-bee8-08e3e643186f 2 0
aa55f848-6a21-45c0-b473-414dd31579f2 2 64.13906259710821
aa55f848-6a21-45c0-b473-414dd31579f2 3 64.29143811327353
61a75d1e-aaf0-49d5-9da4-9bc79f5aeb76 3 0
61a75d1e-aaf0-49d5-9da4-9bc79f5aeb76 2 0
095079b1-d416-4031-960b-1ab7d859233b 1 0
095079b1-d416-4031-960b-1ab7d859233b 0 0
1688e96a-0b69-4ea8-b3e8-cd395ac9e485 3 0
1688e96a-0b69-4ea8-b3e8-cd395ac9e485 2 0
c3aa5b33-608a-426c-92bf-799768c6f12f 2 65.91426020946201
c3aa5b33-608a-426c-92bf-799768c6f12f 3 71.84211446129233
400fe787-a42f-4ae4-98f1-f5159a4500

5.9346
False
9.700799999999983
False
9.586399999999985
False
9.623799999999983
False
9.304799999999986
False
9.243199999999984
False
9.467599999999985
False
9.350999999999985
False
8.734999999999987
False
8.664599999999988
False
7.986999999999991
False
8.15419999999999
False
0.1329999999999996
False
0.8369999999999997
False
9.258599999999984
False
9.744799999999984
False
8.319199999999988
False
0.7027999999999999
False
0.2605999999999996
False
9.760199999999983
2
False
9.733799999999984
False
9.740399999999983
False
9.742599999999983
False
9.630399999999982
False
9.599599999999985
False
9.458799999999984
False
0.2869999999999999
False
9.111199999999986
False
7.711999999999992
False
9.199199999999983
False
8.682199999999987
False
8.959399999999986
False
8.404999999999989
False
7.938599999999991
False
7.81319999999999
False
9.768999999999984
False
8.906599999999987
False
8.121199999999991
False
7.601999999999992
False
6.845199999999994
3
False


KeyboardInterrupt: 

In [ ]:
print(len(champs))
champ_index = 0
champs[champ_index].team.epsilon = 0.0

for q_value in champs[champ_index].team.q_table:
    print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))

wins = 0
for i in range(100):
    env.reset()
    score, win, states = champs[champ_index].replay(env)
    print(win)
    if win:
        print(states)
        wins += 1
    print(score)
print(wins)